Given an array arr of positive integers, consider all binary trees such that:

Each node has either 0 or 2 children;
The values of arr correspond to the values of each leaf in an in-order traversal of the tree.
The value of each non-leaf node is equal to the product of the largest leaf value in its left and right subtree, respectively.
Among all possible binary trees considered, return the smallest possible sum of the values of each non-leaf node. It is guaranteed this sum fits into a 32-bit integer.

A node is a leaf if and only if it has zero children.

 

Example 1:


Input: arr = [6,2,4]
Output: 32
Explanation: There are two possible trees shown.
The first has a non-leaf node sum 36, and the second has non-leaf node sum 32.
Example 2:


Input: arr = [4,11]
Output: 44
 

Constraints:

2 <= arr.length <= 40
1 <= arr[i] <= 15
It is guaranteed that the answer fits into a 32-bit signed integer (i.e., it is less than 231).

In [ ]:
class Solution:
    def mctFromLeafValues(self, arr: List[int]) -> int:
        from functools import lru_cache

        @lru_cache(None)
        def dfs(i, j):
            if i == j:
                return 0  # no non-leaf node
            res = float('inf')
            for k in range(i, j):
                left_max = max(arr[i:k+1])
                right_max = max(arr[k+1:j+1])
                total = dfs(i, k) + dfs(k+1, j) + left_max * right_max
                res = min(res, total)
            return res

        return dfs(0, len(arr) - 1)
# ⏱️ Time and Space Complexity
# Time Complexity: O(n^3)
# Because for each subarray we do O(n) work for max calculation and there are O(n^2) subarrays.

# Space Complexity: O(n^2)
# For memoization cache and recursion stack.

🔍 What is @lru_cache?
@lru_cache stands for Least Recently Used cache.

It memoizes (caches) the results of function calls so that when the same inputs occur again, the cached result is returned instead of recalculating.

Great question!

Let's break down **why** the **monotonic stack greedy approach** works for
**1130. Minimum Cost Tree From Leaf Values**.

---

## 🔥 Goal of the Problem Recap:

We’re given an array of **leaf values**, and we want to build a **binary tree** such that:

* Every node has either 0 or 2 children.
* Leaf nodes are exactly the given array values.
* **Non-leaf node value = product of largest leaf in left & right subtrees.**
* We want to **minimize the sum** of all non-leaf node values.

---

## 💡 Intuition Behind the Greedy + Monotonic Stack Approach

Let’s simplify:

* Imagine merging two leaves creates a non-leaf node with cost `a * b`.
* We must decide **in what order to merge** to minimize the total cost.
* So, we must **avoid big multiplications** early — they blow up the total.

---

## ✅ Greedy Insight

To minimize total cost:

> **Always merge the smallest leaf with its nearest larger neighbor** — because merging two small values gives a smaller product.

That’s where the **monotonic decreasing stack** helps:

* It maintains a decreasing order.
* When we see a value `a` such that `a >= top of stack`, it means we’ve found a neighbor to merge with the smallest one.

---

## 🔄 How it Works (Step-by-step)

Let’s say you encounter:

```
arr = [6, 2, 4]
```

1. Stack = `[∞]`

2. Push 6 → `[∞, 6]`

3. 2 < 6 → push → `[∞, 6, 2]`

4. 4 ≥ 2 → pop 2
   Now, merge 2 with the smaller of its two neighbors:
   → left = 6, right = 4 → merge with `min(6, 4) = 4`
   → `cost = 2 * 4 = 8`

5. Then continue pushing 4 → `[∞, 6, 4]`

6. Cleanup remaining stack:
   Pop 4 → merge with 6 → `4 * 6 = 24`

Total = 8 + 24 = 32 ✅

---

## 🤝 Why It Works (Key Principle)

This is based on a known greedy strategy called the **"Greedy Merge of Least Local Cost"**:

* Whenever you must pair values, always pair the **smallest one** with the **smaller of its neighbors** to prevent large costs later.
* The monotonic stack efficiently finds that smallest one and its best neighbor.

Think of it like this:

* We're always trying to remove the "weakest link" in the chain with the **least cost** before it becomes costly.
* The stack gives us quick access to such "links".

---

## ✅ Guarantees:

* Each element is processed once.
* The greedy decisions are **locally optimal** and **globally minimal** because of the cost structure (product is superlinear).

---

Would you like this same explanation **as you'd say it in an interview**?


In [ ]:
class Solution:
    def mctFromLeafValues(self, arr: list[int]) -> int:
        stack = [float('inf')]
        res = 0

        for a in arr:
            while stack[-1] <= a:
                # maintain the decreasing stack.
                mid = stack.pop()

                # add it to the current res.
                res += mid * min(stack[-1], a)
            
            # maintain the stack.
            stack.append(a)


        # process the remaining elements in the stack.
        while len(stack) > 2:
            res += stack.pop() * stack[-1]

        return res


# tc - O(n)
# sc - O(n)

| Step | Current Num | Stack Before | Action        | Popped | Cost (× min of neighbors) | Total Cost | Stack After             |
| ---- | ----------- | ------------ | ------------- | ------ | ------------------------- | ---------- | ----------------------- |
| 1    | 6           | \[]          | Push          | –      | –                         | 0          | \[6]                    |
| 2    | 2           | \[6]         | Push          | –      | –                         | 0          | \[6, 2]                 |
| 3    | 4           | \[6, 2]      | Pop 2 (4 > 2) | 2      | 2 × min(6, 4) = 8         | 8          | \[6] → push 4 → \[6, 4] |
| 4    | 7           | \[6, 4]      | Pop 4 (7 > 4) | 4      | 4 × min(6, 7) = 24        | 32         | \[6] → push 7 → \[6, 7] |
| 5    | 5           | \[6, 7]      | Push          | –      | –                         | 32         | \[6, 7, 5]              |
| 6    | 1           | \[6, 7, 5]   | Push          | –      | –                         | 32         | \[6, 7, 5, 1]           |



Now process the remaining stack:


| Step | Stack Before  | Action | Popped | Cost (× min of neighbors) | Total Cost | Stack After |
| ---- | ------------- | ------ | ------ | ------------------------- | ---------- | ----------- |
| 7    | \[6, 7, 5, 1] | Pop 1  | 1      | 1 × min(5, ∞) = 5         | 37         | \[6, 7, 5]  |
| 8    | \[6, 7, 5]    | Pop 5  | 5      | 5 × min(7, ∞) = 35        | 72         | \[6, 7]     |
| 9    | \[6, 7]       | Pop 7  | 7      | 7 × min(6, ∞) = 42        | 114        | \[6]        |
